In [ ]:
import sys
import json
import copy
import time
from typing import List, Dict, Any

from PyQt5.QtWidgets import QApplication
from PyQt5.QtCore import QTimer

# 项目路径
SIMWORLD_DIR = r"D:\Projects\Food-Delivery-Bench\SimWorld"
LLM_DELIVERY_DIR = r"D:\Projects\Food-Delivery-Bench\LLM-Delivery"

# 数据文件
ROADS_JSON = r"D:\Projects\Food-Delivery-Bench\Test_Data\test\roads.json"
WORLD_JSON = r"D:\Projects\Food-Delivery-Bench\Test_Data\test\progen_world_enriched.json"
STORE_ITEMS_JSON = r"D:\Projects\Food-Delivery-Bench\LLM-Delivery\input\store_items.json"
FOOD_JSON = r"D:\Projects\Food-Delivery-Bench\LLM-Delivery\input\food.json"
CONFIG_JSON = r"D:\Projects\Food-Delivery-Bench\LLM-Delivery\input\config.json"

# 项目路径注入
sys.path.insert(0, SIMWORLD_DIR)
sys.path.insert(0, LLM_DELIVERY_DIR)

from Base.Map import Map
from Base.DeliveryMan import DeliveryMan, DMAction, DMActionKind, TransportMode
from Base.Timer import VirtualClock
from utils.map_observer import MapObserver
from Base.ActionSpace import parse_action
from Base.Bus import Bus, BusRoute, BusStop
from Base.BusManager import BusManager
from Communicator.Communicator import Communicator

def _load_world_nodes(path: str):
    with open(path, "r", encoding="utf-8") as f:
        return json.load(f).get("nodes", [])
    
def _load_cfg(path: str) -> dict:
    try:
        with open(path, "r", encoding="utf-8") as f:
            data = json.load(f) or {}
        return data
    except FileNotFoundError:
        raise RuntimeError(f"Config file not found: {path}")
    except json.JSONDecodeError as e:
        raise RuntimeError(f"Config JSON parse error in {path}: {e}")

# 1. 加载地图和配置
cfg = _load_cfg(CONFIG_JSON)
clock = VirtualClock(time_scale=1.0)

app = QApplication(sys.argv)

# 2. 创建地图
m = Map(); m.import_roads(ROADS_JSON); m.import_pois(WORLD_JSON)
nodes = _load_world_nodes(WORLD_JSON)

bus_manager = BusManager(clock=clock, waiting_time_s=cfg.get("bus", {}).get("waiting_time_s", 10.0), speed_cm_s=cfg.get("bus", {}).get("speed_cm_s", 1200.0))
    
# 从世界数据加载公交路线
with open(WORLD_JSON, "r", encoding="utf-8") as f:
    world_data = json.load(f)
bus_manager.init_bus_system(world_data)

ue = Communicator(port=9000, ip='127.0.0.1', resolution=(640, 480))

# --- Viewer ---
v = MapObserver(title="Unit Test", clock=clock)
v.draw_map(m, WORLD_JSON, show_bus=True, show_docks=False,
            show_building_links=True, show_road_names=True, plain_mode="pudo")
v.resize(1200, 900); v.show()
# v.attach_order_manager(om)
# v.attach_comms(comms)
v.attach_bus_manager(bus_manager)

# 3. 创建DeliveryMan实例
dm = DeliveryMan(
    agent_id="test_agent",
    city_map=m,
    world_nodes=nodes,
    x=100.0,  # 初始x坐标
    y=100.0,  # 初始y坐标
    mode=TransportMode.SCOOTER,
    clock=clock,
    cfg=copy.deepcopy(cfg)
)

# 4. 初始化必要的组件
dm.bind_viewer(v)  
dm.set_bus_manager(bus_manager) 
dm.set_ue(ue)
dm.bind_simworld()



# 5. 测试特定的action
def test_action(action_text: str):
    """测试单个action"""
    try:
        # 解析action
        action = parse_action(action_text, dm)
        print(f"解析的action: {action}")
        
        # 直接执行action
        dm.enqueue_action(action, allow_interrupt=True)
        
        # 等待action完成（这里需要根据具体action调整）
        time.sleep(1)  # 给action一些时间执行
        
        print(f"Action执行完成")
        
    except Exception as e:
        print(f"Action执行失败: {e}")

time.sleep(20)

# 测试不同的actions
test_actions = [
    "TURN_AROUND(angle=60, direction='left')",
    "STEP_FORWARD()",
    "BOARD_BUS(bus_id=0, target_stop_id=0)",
    "VIEW_BUS_SCHEDULE()",
]

for action_text in test_actions:
    print(f"\n=== 测试Action: {action_text} ===")
    test_action(action_text)